In [1]:
import pandas as pd
import psycopg2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import f1_score, multilabel_confusion_matrix
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import MNLogit

In [29]:
# Carregando o Dataset
df = pd.read_pickle('../storage/pickle/dataframe_amostra_no_dummies0.15.pkl')
df = df.drop_duplicates()
# Tratando dado de idade
df["idade"] = pd.to_numeric(df["idade"])

In [30]:
# Conecta ao banco de dados
conn = psycopg2.connect(
    host="localhost",
    database="previsao-patologias",
    user="postgres",
    password="test"
)

cursor = conn.cursor()
query = "SELECT texto FROM tb_questoes WHERE tipo_resposta = 'binaria';"
cursor.execute(query)

# Pega resultado da query e transforma em lista para realizar iteração
tuplas_resultados = cursor.fetchall()
questoes = [item for tupla in tuplas_resultados for item in tupla]

In [31]:
questoes_binarias = []
for c in questoes:
    nome_coluna_tratado = c.replace(" ", "_").replace(".", "").replace("?", "").lower()
    questoes_binarias.append(nome_coluna_tratado)
    
colunas_dummies = [col for col in df.columns if col not in ['idade', 'patologia', 'severidade'] + questoes_binarias]

df_dummies = pd.get_dummies(df, columns=colunas_dummies, dtype=int)

Separando variáveis dependentes e de testes e criando conjunto de treino e testes

In [32]:
x = df_dummies.drop(columns=['patologia', 'severidade']).astype('int64')
y = df_dummies['patologia']

In [33]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [37]:
params = {'C': 0.001, 'penalty': 'l2'}
regressao_logistica = LogisticRegression(multi_class='multinomial', solver="lbfgs", max_iter=50, random_state=42, n_jobs=-1, **params)

Realizando Fit das Variáveis e Predição das Observações

In [38]:
# Realizando o fit das variáveis
regressao_logistica.fit(X_train, y_train)

# Fazer previsões
y_pred = regressao_logistica.predict(X_test)

/home/guilhermehip/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Utilizando técnica de acurácia comum

In [71]:
# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
classification_report_result = classification_report(y_test, y_pred)
matriz_confusao = confusion_matrix(y_test, y_pred, labels=regressao_logistica.classes_)
f1 = f1_score(y_test, y_pred, average='macro')

# Calcular sensibilidade (recall) e especificidade para cada classe
sensibilidade = np.diag(matriz_confusao) / np.sum(matriz_confusao, axis=1)

# Calcular sensibilidade e especificidade médias (macro)
sensibilidade_media = np.mean(sensibilidade)

# Exibir métricas de desempenho
print(f'Acurácia: {accuracy}')
print('Relatório de Classificação:\n', classification_report_result)
print(f"F1-Score: {f1}")
print(f'Sensibilidade (Recall) Média: {sensibilidade_media}')

/home/guilhermehip/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/guilhermehip/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/guilhermehip/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Acurácia: 0.9270661157024793
Relatório de Classificação:
                                           precision    recall  f1-score   support

     acute copd exacerbation / infection       0.94      1.00      0.97       660
                acute dystonic reactions       1.00      0.99      1.00      1017
                        acute laryngitis       0.85      0.85      0.85       905
                      acute otitis media       0.87      0.94      0.91       978
                   acute pulmonary edema       1.00      1.00      1.00       699
                    acute rhinosinusitis       1.00      0.11      0.20       487
                      allergic sinusitis       0.85      0.99      0.92       879
                             anaphylaxis       1.00      1.00      1.00      1056
                                  anemia       1.00      0.99      1.00      1939
                     atrial fibrillation       1.00      0.99      1.00       807
                               boerhaav

Utilizando K-fold Cross Validation

In [38]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
acuracia_validacao_cruzada = cross_val_score(regressao_logistica, x, y, cv=kfold, scoring='accuracy')

# Exibir a acurácia média e o desvio padrão
print(f'Acurácia (k-fold): {acuracia_validacao_cruzada.mean()}')
print(f'Desvio Padrão da Acurácia (k-fold): {acuracia_validacao_cruzada.std()}')

/home/guilhermehip/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/guilhermehip/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

Acurácia (k-fold): 0.9768740453614955
Desvio Padrão da Acurácia (k-fold): 0.0032105341492067105


In [2]:
# Carregando o Dataset
df = pd.read_pickle('../storage/pickle/dataframe_smote0.10.pkl')
df = df.drop_duplicates()
# Tratando dado de idade
df["idade"] = pd.to_numeric(df["idade"])

In [3]:
variaveis_dependentes = df.drop(columns=['patologia', 'severidade'])
variavel_target = df['patologia']

In [4]:
# Splitando conjuntos de treino e testes
X_train, X_test, y_train, y_test = train_test_split(variaveis_dependentes, variavel_target, test_size=0.2, random_state=42)

In [5]:
params = {'C': 0.001, 'penalty': 'l2'}
regressao_logistica = LogisticRegression(multi_class='multinomial', solver="lbfgs", max_iter=50, random_state=42, n_jobs=-1, **params)

In [6]:
# Realizando o fit das variáveis
regressao_logistica.fit(X_train, y_train)

# Fazer previsões
y_pred = regressao_logistica.predict(X_test)

/home/guilhermehip/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
classification_report_result = classification_report(y_test, y_pred)
matriz_confusao = confusion_matrix(y_test, y_pred, labels=regressao_logistica.classes_)
f1 = f1_score(y_test, y_pred, average='macro')

# Calcular sensibilidade (recall) e especificidade para cada classe
sensibilidade = np.diag(matriz_confusao) / np.sum(matriz_confusao, axis=1)

# Calcular sensibilidade e especificidade médias (macro)
sensibilidade_media = np.mean(sensibilidade)

# Exibir métricas de desempenho
print(f'Acurácia: {accuracy}')
print('Relatório de Classificação:\n', classification_report_result)
print(f"F1-Score: {f1}")
print(f'Sensibilidade (Recall) Média: {sensibilidade_media}')

Acurácia: 0.9825320916184244
Relatório de Classificação:
                                           precision    recall  f1-score   support

     acute copd exacerbation / infection       1.00      1.00      1.00       803
                acute dystonic reactions       1.00      0.96      0.98       781
                        acute laryngitis       0.99      0.99      0.99       829
                      acute otitis media       1.00      0.98      0.99       850
                   acute pulmonary edema       1.00      1.00      1.00       796
                    acute rhinosinusitis       0.94      0.79      0.86       820
                      allergic sinusitis       1.00      0.95      0.98       789
                             anaphylaxis       1.00      1.00      1.00       851
                                  anemia       1.00      1.00      1.00       775
                     atrial fibrillation       1.00      0.98      0.99       797
                               boerhaav